# Random Forest

## Prior Progress
Random Forest 13 did shuffle and cross validation for 70% accuracy.

## Next Task
Bring in RBC data.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-06 14:29:40.933126
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
BASE_DIR='/home/jrm/ShepherdML/TumorII/'
CLASS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
CLASSES=range(4,6)   # load just 2 small classes for testing
CLASSES=range(1,6)   # we only have 1 thru 5 right now
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.10/'
DESCRIBE=False  # just retain the mean over objects in patch (not even the count!)
DESCRIBE=True   # compute stats for every column

In [4]:
print(datetime.datetime.now())
df = None
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        Xi = pd.read_csv(BASE_DIR+CLASS_FILES[i])
        Xi.drop(columns=['PatchNumber'],inplace=True)
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-06 14:29:42.090043


,ObjectNumber_count,ObjectNumber_mean,ObjectNumber_std,ObjectNumber_min,ObjectNumber_25%,ObjectNumber_50%,ObjectNumber_75%,ObjectNumber_max,AreaShape_Area_count,AreaShape_Area_mean,...,Texture_Variance_Hematoxylin_7_02_256_75%,Texture_Variance_Hematoxylin_7_02_256_max,Texture_Variance_Hematoxylin_7_03_256_count,Texture_Variance_Hematoxylin_7_03_256_mean,Texture_Variance_Hematoxylin_7_03_256_std,Texture_Variance_Hematoxylin_7_03_256_min,Texture_Variance_Hematoxylin_7_03_256_25%,Texture_Variance_Hematoxylin_7_03_256_50%,Texture_Variance_Hematoxylin_7_03_256_75%,Texture_Variance_Hematoxylin_7_03_256_max
0,23.0,12.0,6.782330,1.0,6.50,12.0,17.50,23.0,23.0,465.391304,...,1569.195581,2305.155794,23.0,1184.715966,690.835879,261.731230,673.746426,944.021689,1640.900996,2777.568252
1,15.0,8.0,4.472136,1.0,4.50,8.0,11.50,15.0,15.0,375.933333,...,1986.180592,2429.516922,15.0,1761.768466,677.143063,657.512308,1311.381449,1944.654374,2198.298670,2918.261469
2,40.0,20.5,11.690452,1.0,10.75,20.5,30.25,40.0,40.0,526.725000,...,985.091747,1367.235640,40.0,779.622308,341.530535,72.299357,500.097355,716.335046,1030.380592,1560.196385
3,15.0,8.0,4.472136,1.0,4.50,8.0,11.50,15.0,15.0,451.133333,...,1836.272933,4597.420676,15.0,1655.311137,995.211612,826.073262,1185.611583,1374.135493,1712.762472,4874.759263
4,18.0,9.5,5.338539,1.0,5.25,9.5,13.75,18.0,18.0,421.388889,...,2174.083732,2905.217328,18.0,1690.673081,470.364895,719.832593,1333.380895,1815.091217,2061.899879,2342.868103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,2645.755171,5002.663831,7.0,2072.810441,1442.119639,682.306939,946.916962,2007.931412,2641.212717,4643.175378
1191,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,2697.422264,3144.417339,17.0,2120.125661,784.452285,435.222222,1636.718754,2149.951462,2783.133279,3362.958125
1192,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,2196.200883,2971.276407,19.0,1731.620347,639.473204,677.151060,1348.982274,1626.336416,2185.555700,2992.328549
1193,33.0,17.0,9.669540,1.0,9.00,17.0,25.00,33.0,33.0,444.454545,...,1931.452617,2672.847178,33.0,1330.902406,667.821030,479.463374,800.137069,1102.353972,1578.432099,2878.275735


In [5]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

Ok


In [6]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model=RandomForestClassifier()
        self.model.fit(self.Xtr,self.ytr)
    def cross_validation(self,fold=5):
        self.model=RandomForestClassifier()
        cv_scores = cross_val_score(self.model, self.Xtr, self.ytr, cv=fold)
        return cv_scores
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

2022-06-06 14:30:02.472783


In [7]:
print(datetime.datetime.now())
print('Do a one-time train/valid split...')
Xt,Xv,yt,yv = train_test_split(Xtrain, ytrain.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xt.shape,'ytrain',yt.shape,'non-zero:',np.count_nonzero(yt))
print('Xvalid',Xv.shape,'yvalid',yv.shape,'non-zero:',np.count_nonzero(yv))
print('Do a one-time fit and evaluate...')
rf1 = RF_Util()
rf1.set_train(Xt,yt)
rf1.set_validation(Xv,yv)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())

2022-06-06 14:30:02.477252
Do a one-time train/valid split...
Xtrain (22640, 5192) ytrain (22640,) non-zero: 22640
Xvalid (7547, 5192) yvalid (7547,) non-zero: 7547
Do a one-time fit and evaluate...
2022-06-06 14:30:03.011510
2022-06-06 14:31:39.522546
Accuracy: 69.31230952696436
Confusion:
[[1946  490   56    8    0]
 [ 494 2206   76    9   20]
 [ 313  349  639    3    0]
 [ 126  197   43  248    2]
 [  39   79   10    2  192]]


In [8]:
Xt,Xv,yt,yv=None,None,None,None
rf1=None

In [9]:
print(datetime.datetime.now())
print("Shuffle...")
# Shuffle (happens automatically with cross validation but not with fit())
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-06 14:31:40.169712
Shuffle...


In [10]:
print(datetime.datetime.now())
print('Cross-validation...')
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
cv_scores = rf2.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-06 14:31:40.551123
Cross-validation...
[0.69145412 0.69973501 0.69173431 0.69372205 0.70101044]
mean 0.6955 +/- 0.0040


In [11]:
rf2=None

In [12]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf3 = RF_Util()
rf3.set_train(Xtrain,ytrain)
rf3.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf3.important_features()

2022-06-06 14:40:20.055174
Re-train on full training set...
2022-06-06 14:42:33.423342
...and rank the features by importance.


In [13]:
pd.set_option('display.max_rows', 25)
top.loc[:25]

,0,1
0,0.001985,Texture_Correlation_Eosin_3_00_256_mean
1,0.001580,Texture_Correlation_Eosin_3_00_256_75%
2,0.001519,Texture_Correlation_Eosin_3_00_256_25%
3,0.001485,Texture_Correlation_Eosin_3_00_256_50%
4,0.001468,Texture_InverseDifferenceMoment_Hematoxylin_3_...
...,...,...
21,0.001136,Texture_SumAverage_Eosin_4_02_256_mean
22,0.001120,Granularity_1_Hematoxylin_25%
23,0.001119,Texture_SumAverage_Eosin_5_00_256_25%
24,0.001109,Texture_SumAverage_Eosin_3_01_256_50%


In [14]:
print(datetime.datetime.now())

2022-06-06 14:42:33.451983
